In [ ]:
La empresa ADL se dedica a la venta de productos a nivel regional en los Estados Unidos,
con equipos de ventas que operan en diferentes estados. La compañía ha establecido un
sistema de comisiones para incentivar a sus equipos de ventas a preferir la venta de
productos de alto margen antes que los de bajo margen de utilidad.
Como Analista de Datos, le solicitan analizar información sobre las ventas realizadas por
los equipos de ventas de ADL, incluyendo detalles sobre los productos vendidos, los
clientes, las fechas de los pedidos y los plazos de entrega.
Para resolver este desafío deberás cargar datos desde Hojas de Excel del archivo
US_Regional_Sales_Data.xlsx. En base a esto, debe generar una visualización de datos
como la que se muestra en las hojas Reporte1 y Reporte2 del archivo reporte.xlsx.
Para cumplir con lo solicitado, puedes aplicar los siguientes pasos:

In [ ]:
Carga los datos de cada pestaña del archivo US_Regional_Sales_Data.xlsx en un
DataFrame independiente. (hint: Puedes obtener los nombres de las hojas usando
pd.ExcelFile(archivo).sheet_names)

In [9]:
#como siempre importamos pandas 
import pandas as pd

#y cargamos los datos de cada pestaña del archivo US_Regional_Sales_Data.xlsx en un DataFrame independiente
xls = pd.ExcelFile("US_Regional_Sales_Data.xlsx")
sheet_names = xls.sheet_names
print(sheet_names)

['Sales Orders Sheet', 'Customers Sheet', 'Store Locations Sheet', 'Products Sheet', 'Sales Team Sheet']


In [10]:
#como ya tenemos los nombres de las hojas procedemos a crearlos de forma independiente
df_orders = pd.read_excel(xls, sheet_names[0])
df_products = pd.read_excel(xls, sheet_names[1])
df_customers = pd.read_excel(xls, sheet_names[2])

# Verificar los primeros registros de cada DataFrame
print(df_orders.head())
print(df_products.head())
print(df_customers.head())

   OrderNumber Sales Channel WarehouseCode ProcuredDate  OrderDate   ShipDate  \
0  SO - 000101      In-Store  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-14   
1  SO - 000102        Online  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-22   
2  SO - 000103   Distributor  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-21   
3  SO - 000104     Wholesale  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-02   
4  SO - 000105   Distributor  WARE-NMK1003   2018-04-10 2018-05-31 2018-06-16   

  DeliveryDate CurrencyCode  _SalesTeamID  _CustomerID  _StoreID  _ProductID  \
0   2018-06-19          USD             6           15       259          12   
1   2018-07-02          USD            14           20       196          27   
2   2018-07-01          USD            21           16       213          16   
3   2018-06-07          USD            28           48       107          23   
4   2018-06-26          USD            22           49       111          26   

   Order Quantity  Discount Appl

In [18]:
#Cruzamos todos los DataFrames usando validación many_to_one
df_base = pd.merge(df_orders, df_products, how="left", left_on="_ProductID", right_on="_CustomerID")
df_base = pd.merge(df_base, df_customers, how="left", left_on="_StoreID", right_on="_StoreID")

# Verificamos los primeros registros de df_base
print(df_base.head())



   OrderNumber Sales Channel WarehouseCode ProcuredDate  OrderDate   ShipDate  \
0  SO - 000101      In-Store  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-14   
1  SO - 000102        Online  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-22   
2  SO - 000103   Distributor  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-21   
3  SO - 000104     Wholesale  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-02   
4  SO - 000105   Distributor  WARE-NMK1003   2018-04-10 2018-05-31 2018-06-16   

  DeliveryDate CurrencyCode  _SalesTeamID  _CustomerID_x  ...  Type  Latitude  \
0   2018-06-19          USD             6             15  ...  Town  40.64092   
1   2018-07-02          USD            14             20  ...  City  38.98223   
2   2018-07-01          USD            21             16  ...  City  42.27087   
3   2018-06-07          USD            28             48  ...  City  41.30815   
4   2018-06-26          USD            22             49  ...  City  41.55815   

   Longitude  AreaCode  Po

Agrega las siguientes columnas a df_ base:
Columna Cálculo
ProcurementDays OrderDate-ProcuredDate
ShippingDays ShipDate-OrderDate

In [21]:
# primero debemos verificar y convertir las fechas a datetime ya que usamos el procureDate
df_base['ProcuredDate'] = pd.to_datetime(df_base['ProcuredDate'])
df_base['OrderDate'] = pd.to_datetime(df_base['OrderDate'])
df_base['ShipDate'] = pd.to_datetime(df_base['ShipDate'])
df_base['DeliveryDate'] = pd.to_datetime(df_base['DeliveryDate'])

# calculamos ProcurementDays, ShippingDays, DeliveryDays y CustomerDays
df_base['ProcurementDays'] = (df_base['OrderDate'] - df_base['ProcuredDate']).dt.days
df_base['ShippingDays'] = (df_base['ShipDate'] - df_base['OrderDate']).dt.days
df_base['DeliveryDays'] = (df_base['DeliveryDate'] - df_base['ShipDate']).dt.days
df_base['CustomerDays'] = df_base['ShippingDays'] + df_base['DeliveryDays']

# Verificar los primeros registros de df_base con las nuevas columnas
print(df_base.head())


   OrderNumber Sales Channel WarehouseCode ProcuredDate  OrderDate   ShipDate  \
0  SO - 000101      In-Store  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-14   
1  SO - 000102        Online  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-22   
2  SO - 000103   Distributor  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-21   
3  SO - 000104     Wholesale  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-02   
4  SO - 000105   Distributor  WARE-NMK1003   2018-04-10 2018-05-31 2018-06-16   

  DeliveryDate CurrencyCode  _SalesTeamID  _CustomerID_x  ...  Population  \
0   2018-06-19          USD             6             15  ...      213776   
1   2018-07-02          USD            14             20  ...      186515   
2   2018-07-01          USD            21             16  ...      117070   
3   2018-06-07          USD            28             48  ...      130322   
4   2018-06-26          USD            22             49  ...      108802   

   Household Income  Median Income  Land Area  Wat

Agrega una nueva columna discreta al df_base, con el nombre CustomerDaysInterval
que clasifique los valores de la columna CustomerDays en los siguientes intervalos:
○ 0 to 15 days
○ 15 to 30 days
○ 30 to 45 days
○ 45 to 60 days
○ 60 to 75 days
○ 75 to 90 days


In [22]:
# aquidebemos definir los pasos de los intervalos segun lo de la guia ademas los etiquetaremos
intervalos = [0, 15, 30, 45, 60, 75, 90]
etiquetas = ['0 to 15 days', '15 to 30 days', '30 to 45 days', '45 to 60 days', '60 to 75 days', '75 to 90 days']

# Agregamos la nueva columna CustomerDaysInterval al df_base
df_base['CustomerDaysInterval'] = pd.cut(df_base['CustomerDays'], bins=intervalos, labels=etiquetas, right=False)

# Verificamos los primeros registros de df_base con la nueva columna CustomerDaysInterval
print(df_base.head())


   OrderNumber Sales Channel WarehouseCode ProcuredDate  OrderDate   ShipDate  \
0  SO - 000101      In-Store  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-14   
1  SO - 000102        Online  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-22   
2  SO - 000103   Distributor  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-21   
3  SO - 000104     Wholesale  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-02   
4  SO - 000105   Distributor  WARE-NMK1003   2018-04-10 2018-05-31 2018-06-16   

  DeliveryDate CurrencyCode  _SalesTeamID  _CustomerID_x  ...  \
0   2018-06-19          USD             6             15  ...   
1   2018-07-02          USD            14             20  ...   
2   2018-07-01          USD            21             16  ...   
3   2018-06-07          USD            28             48  ...   
4   2018-06-26          USD            22             49  ...   

   Household Income  Median Income  Land Area  Water Area         Time Zone  \
0             68789          80327  1354813

Utiliza una tabla pivote para mostrar el conteo de órdenes agrupando por Sales
Team en las filas y CustomerDaysInterval en las columnas. Este reporte debe
cuadrar con los datos de la pestaña Reporte1 del archivo reportes.xlsx


In [23]:
# Crear una tabla pivote
pivot_table = pd.pivot_table(df_base, 
                              index='_SalesTeamID', 
                              columns='CustomerDaysInterval', 
                              values='OrderNumber', 
                              aggfunc='count', 
                              fill_value=0)

# Ordnamos las columnas según el orden especificado en las etiquetas
pivot_table = pivot_table.reindex(columns=etiquetas)

# visualizaremos la tabla pivote
print(pivot_table)

CustomerDaysInterval  0 to 15 days  15 to 30 days  30 to 45 days  \
_SalesTeamID                                                       
1                               71            176             55   
2                               84            130             32   
3                               81            162             53   
4                               86            152             54   
5                               87            151             45   
6                               79            148             38   
7                               75            175             53   
8                               92            169             54   
9                               76            166             43   
10                              74            141             45   
11                              74            166             49   
12                              85            184             45   
13                             103            18

In [25]:
#comprobaremos que cuadren los archivos y usaremos un verificador sencillo de python "if" en los documentos
df_reporte = pd.read_excel('reportes.xlsx', sheet_name='Reporte1')

# visualizamos la tabla pivote
print("Tabla Pivote:")
print(pivot_table)

# visualizamos los datos de Reporte1
print("\nDatos de Reporte1:")
print(df_reporte)

# Verificamos si las tablas coinciden
if pivot_table.equals(df_reporte):
    print("\nLas tablas coinciden.")
else:
    print("\nno coindicen los datos.")


Tabla Pivote:
CustomerDaysInterval  0 to 15 days  15 to 30 days  30 to 45 days  \
_SalesTeamID                                                       
1                               71            176             55   
2                               84            130             32   
3                               81            162             53   
4                               86            152             54   
5                               87            151             45   
6                               79            148             38   
7                               75            175             53   
8                               92            169             54   
9                               76            166             43   
10                              74            141             45   
11                              74            166             49   
12                              85            184             45   
13                             103

Agrega la siguiente columna calculada a la base:
KPI Cálculo
GrossMargin OrderQuantity*(UnitPrice*(1-Discount Applied)-UnitCost)
Visión de Negocio: El margen bruto es un indicador financiero que muestra la diferencia entre
los ingresos generados por la venta de un producto o servicio y los costos directos asociados
con su producción o adquisición.
7. En esta parte debes aplicar un porcentaje a la columna GrossMargin, pero ese
porcentaje dependerá del intervalo en que se encuentre GrossMargin

In [26]:
# Calculamos el GrossMargin y agregarlo como una nueva columna a df_base
df_base['GrossMargin'] = df_base['Order Quantity'] * ((df_base['Unit Price'] * (1 - df_base['Discount Applied'])) - df_base['Unit Cost'])

# Mostraremos las primeras filas de df_base para verificar la adición de la nueva columna
print(df_base.head())


   OrderNumber Sales Channel WarehouseCode ProcuredDate  OrderDate   ShipDate  \
0  SO - 000101      In-Store  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-14   
1  SO - 000102        Online  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-22   
2  SO - 000103   Distributor  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-21   
3  SO - 000104     Wholesale  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-02   
4  SO - 000105   Distributor  WARE-NMK1003   2018-04-10 2018-05-31 2018-06-16   

  DeliveryDate CurrencyCode  _SalesTeamID  _CustomerID_x  ...  Median Income  \
0   2018-06-19          USD             6             15  ...          80327   
1   2018-07-02          USD            14             20  ...          72463   
2   2018-07-01          USD            21             16  ...          55990   
3   2018-06-07          USD            28             48  ...          37192   
4   2018-06-26          USD            22             49  ...          40467   

   Land Area  Water Area        

En esta parte debes aplicar un porcentaje a la columna GrossMargin, pero ese
porcentaje dependerá del intervalo en que se encuentre GrossMargin.
KPI Cálculo
CommissionsPercentage % de comisión según la tabla de comisiones
hint: Utiliza pd.cut con labels numéricas para que la columna resultante sea de tipo float.

In [29]:
#como utilizaremos numpy lo debemos importar
import numpy as np
#definimos los intervalos y los porcentajes de comisión correspondientes
intervalos = [-np.inf, 0, 500, 1000, 1500, np.inf]
porcentajes = [0, 0.02, 0.04, 0.06, 0.08]  # Porcentajes de comisión para cada intervalo

#creamos la columna CommissionsPercentage usando pd.cut
df_base['CommissionsPercentage'] = pd.cut(df_base['GrossMargin'], bins=intervalos, labels=porcentajes, include_lowest=True).astype(float)

#visualizamos las primeras filas de df_base para verificar la adición de la nueva columna
print(df_base.head())


   OrderNumber Sales Channel WarehouseCode ProcuredDate  OrderDate   ShipDate  \
0  SO - 000101      In-Store  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-14   
1  SO - 000102        Online  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-22   
2  SO - 000103   Distributor  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-21   
3  SO - 000104     Wholesale  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-02   
4  SO - 000105   Distributor  WARE-NMK1003   2018-04-10 2018-05-31 2018-06-16   

  DeliveryDate CurrencyCode  _SalesTeamID  _CustomerID_x  ...  Land Area  \
0   2018-06-19          USD             6             15  ...  135481314   
1   2018-07-02          USD            14             20  ...  194562941   
2   2018-07-01          USD            21             16  ...   72727009   
3   2018-06-07          USD            28             48  ...   48407373   
4   2018-06-26          USD            22             49  ...   73880017   

   Water Area         Time Zone  ProcurementDays  Shippi

 Calcula el monto de la comisión, utilizando la Tabla del Anexo.
KPI Cálculo
CommissionsAmount GrossMargin*CommissionsPercentage

In [30]:
#definimos la tabla de comisiones
tabla_comisiones = {
    0: 0.0,
    0.02: 0.05,
    0.04: 0.07,
    0.06: 0.1,
    0.08: 0.15
}

#calculamos el monto de la comisión
df_base['CommissionsAmount'] = df_base['GrossMargin'] * df_base['CommissionsPercentage'].map(tabla_comisiones)

#visualizamos las primeras filas de df_base para verificar la adición de la nueva columna
print(df_base.head())


   OrderNumber Sales Channel WarehouseCode ProcuredDate  OrderDate   ShipDate  \
0  SO - 000101      In-Store  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-14   
1  SO - 000102        Online  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-22   
2  SO - 000103   Distributor  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-21   
3  SO - 000104     Wholesale  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-02   
4  SO - 000105   Distributor  WARE-NMK1003   2018-04-10 2018-05-31 2018-06-16   

  DeliveryDate CurrencyCode  _SalesTeamID  _CustomerID_x  ...  Water Area  \
0   2018-06-19          USD             6             15  ...   160302131   
1   2018-07-02          USD            14             20  ...     1309517   
2   2018-07-01          USD            21             16  ...     2254528   
3   2018-06-07          USD            28             48  ...     3739487   
4   2018-06-26          USD            22             49  ...     1086045   

          Time Zone  ProcurementDays  ShippingDays

 Calcula la comisión sobre el margen bruto.
KPI Cálculo
NetMargin GrossMargin-CommissionsAmount

In [31]:
#calculamos el margen neto
df_base['NetMargin'] = df_base['GrossMargin'] - df_base['CommissionsAmount']

#visualizamos las primeras filas de df_base para verificar la adición de la nueva columna
print(df_base.head())


   OrderNumber Sales Channel WarehouseCode ProcuredDate  OrderDate   ShipDate  \
0  SO - 000101      In-Store  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-14   
1  SO - 000102        Online  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-22   
2  SO - 000103   Distributor  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-21   
3  SO - 000104     Wholesale  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-02   
4  SO - 000105   Distributor  WARE-NMK1003   2018-04-10 2018-05-31 2018-06-16   

  DeliveryDate CurrencyCode  _SalesTeamID  _CustomerID_x  ...  \
0   2018-06-19          USD             6             15  ...   
1   2018-07-02          USD            14             20  ...   
2   2018-07-01          USD            21             16  ...   
3   2018-06-07          USD            28             48  ...   
4   2018-06-26          USD            22             49  ...   

          Time Zone  ProcurementDays  ShippingDays  DeliveryDays  \
0  America/New York              151            14    

Utilizando una tabla pivote, muestra la suma de los valores GrossMargin, NetMargin y
CommissionAmount agrupando por Sales Team en las filas. Este reporte debe cuadrar con
los datos de la pestaña Reporte2 del archivo reportes.xlsx
Anexo
Tabla de comisiones
Intervalo Comisión
GrossMargin entre $0 y $100 5%
GrossMargin entre $100 y $1000 10%
GrossMargin entre $1000 y $10000 15%
GrossMargin entre $10000 y $100000 20%

In [34]:
#primero que todo debemos definir la tabla de comisones
comisiones = {
    (0, 100): 0.05,
    (100, 1000): 0.1,
    (1000, 10000): 0.15,
    (10000, 100000): 0.2
}

#utilizaremos una función para calcular el porcentaje de comisión basado en el margen bruto (def)
def calcular_comision(margen_bruto):
    for rango, comision in comisiones.items():
        if rango[0] < margen_bruto <= rango[1]:
            return comision
    return 0

# Calcular la comisión para cada fila
df_base['CommissionsPercentage'] = df_base['GrossMargin'].apply(calcular_comision)

# Calcular el monto de la comisión
df_base['CommissionsAmount'] = df_base['GrossMargin'] * df_base['CommissionsPercentage']

# Crear una tabla pivote para mostrar la suma de GrossMargin, NetMargin y CommissionsAmount por Sales Team
reporte_pivote = pd.pivot_table(df_base, index='_SalesTeamID', values=['GrossMargin', 'NetMargin', 'CommissionsAmount'], aggfunc='sum')

# Mostrar el reporte pivote
print(reporte_pivote)


              CommissionsAmount  GrossMargin      NetMargin
_SalesTeamID                                               
1                 126637.207525  779095.1955  663811.722885
2                 112940.122825  675453.5990  572948.561355
3                 126660.436425  788587.7555  674720.085680
4                 122565.707975  781608.7005  670144.879795
5                 116111.480725  707156.7930  604994.542095
6                 108049.111100  661726.5385  566126.714485
7                 135986.295400  854878.0245  731834.218930
8                 136852.255325  839150.7460  717698.101990
9                 114204.905275  702157.8225  599825.380205
10                126216.945000  781391.5870  668318.266165
11                135806.234575  840321.1690  718130.605750
12                123843.212050  774781.4675  662913.071315
13                125832.140550  794225.3700  678803.604025
14                114224.889700  705564.6050  603599.242640
15                122487.895850  773027.